In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import nltk
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
import gc

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
print("Text Results: ")

dataset1 = np.array(pd.read_csv('/content/drive/My Drive/Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('/content/drive/My Drive/Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('/content/drive/My Drive/Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


countPos = 0
dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))
def checkPosNeg(dataset, index):
    for i in range(0, len(dataset)):
        if(dataset[i][0] == index):
            return dataset[i][1]
    return 0

Data = []
Y = []
Data_test = []
Y_test = []
index = -1
for i in range(0, len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    if(val == 0 and countPos>38):
        continue
    Y.append(val)
    index+=1
    if(Y[index] == 0):
        countPos+=1

    
    fileName = "/content/drive/My Drive/Dataset/train_data/" + str(int(dataset3[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])


for i in range(0, len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    if(val == 0):
        continue
    Y.append(val)
    fileName = "/content/drive/My Drive/Dataset/dev_data/" + str(int(dataset1[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])
    
for i in range(0, len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    fileName = "/content/drive/My Drive/Dataset/test_data/" + str(int(dataset2[i][0])) + "_TRANSCRIPT.csv"
    Data_test.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])
Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)
    
for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)
        
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/Dataset/GoogleNews-vectors-negative300.bin.gz', binary=True)

stop_words = set(stopwords.words('english'))
def remove_StopWOrds(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    return filtered_sentence

def checkAcc(Y_pred, Y_test):
    correct = 0
    for i in range(len(Y_pred)):
        if(Y_pred[i] == Y_test[i]):
            correct+=1
    
    return float(correct)/len(Y_pred)


max_num_words = 15
max_num_sentence = 200

#train_data
finalMatrix = np.zeros((Data2.shape[0], max_num_sentence, max_num_words))
print(finalMatrix.shape)
for k in range(Data2.shape[0]):
    for i in range(min(max_num_sentence, len(Data2[k]))):
    	try:
    	  sentence = Data2[k][i].split(" ")
    	except:
    	  continue
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			finalMatrix[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X = np.zeros((len(Data2), max_num_sentence*max_num_words))
for i in range(len(finalMatrix)):
    X[i] = finalMatrix[i].flatten().reshape(1, -1)
 
#Test_data
finalMatrix_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words))
print(finalMatrix_test.shape)
for k in range(Data2_test.shape[0]):
    for i in range(min(max_num_sentence, len(Data2_test[k]))):
    	try:
    	  sentence = Data2_test[k][i].split(" ")
    	except:
    	  continue
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			finalMatrix_test[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X_test = np.zeros((len(Data2_test), max_num_sentence*max_num_words))
for i in range(len(finalMatrix_test)):
    X_test[i] = finalMatrix_test[i].flatten().reshape(1, -1)

Data2 = []
Data2_test = []
gc.collect()          
from sklearn.ensemble import RandomForestClassifier
clf1 = RandomForestClassifier()
clf1.fit(X,Y)
Y_pred_rf_1 = clf1.predict(X_test)
print("Random forests: ")
print(classification_report(Y_test,Y_pred_rf_1))
print("Train: "+ str(clf1.score(X,Y)))
print("Test: " + str(clf1.score(X_test,Y_test)))

from sklearn.svm import SVC

classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X, Y)

Y_pred_svm_1 = classifier.predict(X_test)
print("SVM: ")
print(classification_report(Y_test,Y_pred_svm_1))
print("Train: "+ str(classifier.score(X,Y)))
print("Test: " + str(classifier.score(X_test,Y_test)))

y_pred_train_svm_1 = classifier.predict(X)
y_pred_train_rf_1 = clf1.predict(X)


#Audio Features 
print("Audio Results: ")
import numpy as np
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
def makedata(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    X = np.average(X, axis = 0)
    X = np.array(X.reshape(1, -1))
    return X
 
X_train = np.zeros((1,74))
Y_train = []

X_test = np.zeros((1,74))
#Y_test = []

for i in range(len(dataset2)):
    data = pd.read_csv('/content/drive/My Drive/Dataset/test_data/'+str(int(dataset2[i][0]))+"_COVAREP.csv",header = None)
    X_temp = data.iloc[:,:].values
    X_temp = makedata(X_temp)
    X_test = np.concatenate((X_test,X_temp),0)
    #Y_test.append(dataset2[i][1])
    

X_test = np.delete(X_test,0,0)
#Y_test = np.array(Y_test)
count_0 = 0
count_1 = 0
for i in range(len(dataset3)):
    data = pd.read_csv('/content/drive/My Drive/Dataset/train_data/'+str(int(dataset3[i][0]))+"_COVAREP.csv",header = None)
    if(dataset3[i][1] == 0):
      count_0 +=1
      if(count_0<=39):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dataset3[i][1])
    else:
      count_1 +=1
      if(count_1<=28):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dataset3[i][1])

count_0 = 0
count_1 = 0    

for i in range(len(dataset1)):
    data = pd.read_csv('/content/drive/My Drive/Dataset/dev_data/'+str(int(dataset1[i][0]))+"_COVAREP.csv",header = None)
    if(dataset1[i][1] == 0):
      continue
    else:
      count_1+= 1
      if(count_1<=11):
        X_temp = data.iloc[:,:].values
        X_temp = makedata(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        Y_train.append(dataset1[i][1])
    

X_train = np.delete(X_train,0,0)
Y_train = np.array(Y_train)
clf1 = RandomForestClassifier()
clf1.fit(X_train,Y_train)
Y_pred_rf_2 = clf1.predict(X_test)
print("Random forests: ")
print(classification_report(Y_test,Y_pred_rf_2))
print("Train: "+ str(clf1.score(X_train,Y_train)))
print("Test: " + str(clf1.score(X_test,Y_test)))


from sklearn.svm import SVC


classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, Y_train)
Y_pred_svm_2 = classifier.predict(X_test)
print("SVM: ")
print(classification_report(Y_test,Y_pred_svm_2))
print("Train: "+ str(classifier.score(X_train,Y_train)))
print("Test: " + str(classifier.score(X_test,Y_test)))

y_pred_train_svm_2 = classifier.predict(X_train)
y_pred_train_rf_2 = clf1.predict(X_train)


Text Results: 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


(37, 200, 15)
(16, 200, 15)
Random forests: 
              precision    recall  f1-score   support

           0       0.81      1.00      0.90        13
           1       0.00      0.00      0.00         3

    accuracy                           0.81        16
   macro avg       0.41      0.50      0.45        16
weighted avg       0.66      0.81      0.73        16

Train: 1.0
Test: 0.8125
SVM: 
              precision    recall  f1-score   support

           0       0.81      1.00      0.90        13
           1       0.00      0.00      0.00         3

    accuracy                           0.81        16
   macro avg       0.41      0.50      0.45        16
weighted avg       0.66      0.81      0.73        16

Train: 0.972972972972973
Test: 0.8125
Audio Results: 


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Random forests: 
              precision    recall  f1-score   support

           0       0.81      1.00      0.90        13
           1       0.00      0.00      0.00         3

    accuracy                           0.81        16
   macro avg       0.41      0.50      0.45        16
weighted avg       0.66      0.81      0.73        16

Train: 1.0
Test: 0.8125
SVM: 
              precision    recall  f1-score   support

           0       0.81      1.00      0.90        13
           1       0.00      0.00      0.00         3

    accuracy                           0.81        16
   macro avg       0.41      0.50      0.45        16
weighted avg       0.66      0.81      0.73        16

Train: 0.8378378378378378
Test: 0.8125


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
# Combine
print("Combined Results: ") 
X_2 = np.column_stack((y_pred_train_svm_1,y_pred_train_svm_2))
from sklearn.svm import SVC
classifier2 = SVC(kernel = 'rbf', random_state = 0, gamma = 'auto')
classifier2.fit(X_2, Y_train)
X_3 = np.column_stack(Y_pred_svm_1,Y_pred_svm_2))
y_pred_combine = classifier2.predict(X_3)
print(classification_report(Y_test, y_pred_combine))

Combined Results: 
              precision    recall  f1-score   support

           0       0.81      1.00      0.90        13
           1       0.00      0.00      0.00         3

    accuracy                           0.81        16
   macro avg       0.41      0.50      0.45        16
weighted avg       0.66      0.81      0.73        16



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
